# 🧠 MRI Image Reconstruction - Practical Exercise
## Formal and Hybrid Methods for Medical Imaging Course

**Lesson Objectives:**
- Understand the physical principles of magnetic resonance
- Implement k-space acquisition and reconstruction
- Simulate realistic MRI sequences
- Analyze the FID (raw data)
- Reconstruct k-space from FID
- Reconstruct the image from k-space

---


## Module 1: Setup and Library Import

Let's install and import all necessary libraries for MRI reconstruction.


In [ ]:
# Library installation (uncomment if necessary)
# !pip install numpy scipy matplotlib scikit-image

import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy.fft import fft2, ifft2, fftshift, ifftshift
from skimage import data, transform, filters
import warnings
warnings.filterwarnings('ignore')

# Plot configuration
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12

# Calculate image quality metrics
def calculate_mri_metrics(original, reconstructed):
    """Calculate quality metrics for MRI images"""
    mse = np.mean((original - reconstructed) ** 2)
    psnr = 20 * np.log10(1.0 / np.sqrt(mse)) if mse > 0 else float('inf')

    # Simplified SSIM
    mu1, mu2 = np.mean(original), np.mean(reconstructed)
    sigma1, sigma2 = np.var(original), np.var(reconstructed)
    sigma12 = np.mean((original - mu1) * (reconstructed - mu2))

    c1, c2 = 0.01, 0.03
    ssim = ((2 * mu1 * mu2 + c1) * (2 * sigma12 + c2)) / \
           ((mu1**2 + mu2**2 + c1) * (sigma1 + sigma2 + c2))

    return {
        'MSE': mse,
        'PSNR': psnr,
        'SSIM': ssim
    }

print("✅ Libraries imported successfully!")
print("🧠 Ready for MRI reconstruction!")
print("📊 calculate_mri_metrics function defined globally!")


## Module 2: Creating MRI Phantoms

Let's create realistic phantoms that simulate T2-weighting of MRI images.


In [ ]:
def create_mri_phantom_t2(size=256):
    """
    Create a realistic T2-weighted MRI phantom
    """
    phantom = np.zeros((size, size))
    center = size // 2

    # Coordinate grid
    y, x = np.ogrid[:size, :size]

    # 🧠 Brain tissue simulation

    # 1. Cerebrospinal fluid (CSF) - high T2 signal, low T1
    csf_mask = (x - center)**2 + (y - center)**2 < (size//4)**2         # circular mask
    phantom[csf_mask] = 0.9  # High T2 signal -> water in general has a very long T2 time

    # 2. Gray matter - intermediate signal
    gray_mask = (x - center)**2 + (y - center)**2 < (size//6)**2
    phantom[gray_mask] = 0.6

    # 3. White matter - lower signal
    white_mask = (x - center)**2 + (y - center)**2 < (size//8)**2
    phantom[white_mask] = 0.4

    # 4. Lateral ventricles
    vent1_mask = (x - center - 30)**2 + (y - center - 20)**2 < 15**2
    vent2_mask = (x - center + 30)**2 + (y - center - 20)**2 < 15**2
    phantom[vent1_mask] = 0.95
    phantom[vent2_mask] = 0.95

    # 5. Multiple small vessels more realistic (very low signal)
    vessel1_mask = (np.abs(x - center + 15) < 1) & (y > center - 10) & (y < center + 10)
    phantom[vessel1_mask] = 0.1

    # Vessel 2
    vessel2_mask = (np.abs(x - center - 15) < 1) & (y > center - 15) & (y < center + 5)
    phantom[vessel2_mask] = 0.1

    # Add some realistic texture
    noise = np.random.normal(0, 0.02, (size, size))
    phantom += noise

    # Normalize between 0 and 1
    phantom = np.clip(phantom, 0, 1)

    return phantom

# Create the phantom
phantom_mri = create_mri_phantom_t2(256)

# Visualize
plt.figure(figsize=(10, 5))
plt.imshow(phantom_mri, cmap='gray', vmin=0, vmax=1)
plt.colorbar(label='Signal Intensity')
plt.title('MRI Phantom - Brain Tissues')
plt.xlabel('Pixel')
plt.ylabel('Pixel')

# Add legend
legend_elements = [
    plt.Rectangle((0,0),1,1, facecolor='white', alpha=0.9, label='CSF (High T2)'),
    plt.Rectangle((0,0),1,1, facecolor='gray', alpha=0.6, label='Gray Matter'),
    plt.Rectangle((0,0),1,1, facecolor='darkgray', alpha=0.4, label='White Matter'),
    plt.Rectangle((0,0),1,1, facecolor='black', alpha=0.1, label='Blood Vessels')
]
plt.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

print(f"✅ MRI Phantom created: {phantom_mri.shape}")
print(f"📊 Intensity range: {phantom_mri.min():.3f} - {phantom_mri.max():.3f}")

### 📉 Exercise 1: Renormalize the T2*-weighted image ###

#### In this exercise we are simulating a simple MRI acquisition without a specific sequence -> the FID will be the signal directly acquired by the receiving coil, without refocusing pulses -> since there is no refocusing after the 90° RF pulse we must also consider field inhomogeneities -> we consider a time that accounts for this, namely T2* (smaller than T2)

In [ ]:
def create_t2_star_map(phantom, base_t2_star=15e-3, max_t2_star=35e-3):
    """Create a realistic T2* map based on tissues"""
    t2_star_map = base_t2_star + (phantom * (max_t2_star - base_t2_star))

    # Adjust for specific tissues
    t2_star_map = np.where(phantom > 0.8, 80e-3, t2_star_map)  # CSF: medium-high T2*
    t2_star_map = np.where(phantom < 0.2, 20e-3, t2_star_map)  # Vessels: very low T2*

    return t2_star_map

# Create the phantom
print("\n" + "="*60)
print("Exercise 1: CREATING T2-WEIGHTED PHANTOM")
print("="*60)

phantom = create_mri_phantom_t2(256)
t2_star_map = create_t2_star_map(phantom)

# Visualize
plt.figure(figsize=(7, 5))
plt.imshow(t2_star_map * 1000, cmap='viridis')
plt.colorbar(label='T2* (ms)')
plt.title('T2* Map (ms)')
plt.axis('off')
plt.tight_layout()
plt.show()

print(f"✅ Phantom created: {phantom.shape}")
print(f"📊 Intensity range: {phantom.min():.3f} - {phantom.max():.3f}")
print(f"🕰️ T2* range: {t2_star_map.min()*1000:.1f} - {t2_star_map.max()*1000:.1f} ms")

## Module 3: FID Simulation (raw signal) ##
#### In this module we simulate a raw FID signal acquisition: physically it is a continuous analog signal so in this part the ADC phase will also be included, i.e. analog/digital conversion done with sampling. The sampling phase can be customized as desired by choosing the number of samples to consider per sampling interval (i.e. n_samples per dwell_time)

In [ ]:
def simulate_fid_realistic(phantom, t2_star_map, te_time=100e-3, dwell_time=10e-6, n_samples=512, noise_level=0.01):
    """
    Simulate realistic FID for each pixel

    Args:
        phantom: Proton intensity (density)
        t2_star_map: T2* time map
        te_time: Echo time (s)
        dwell_time: Time between samples (s)
        n_samples: Number of time samples
        noise_level: Noise level

    Returns:
        fid_complex: Complex FID signal [h, w, time]
        time_points: Time array
    """
    print("🔬 FID SIMULATION (Free Induction Decay)")
    print("-" * 40)

    h, w = phantom.shape
    time_points = np.arange(n_samples) * dwell_time

    # Pre-allocate FID array
    fid_complex = np.zeros((h, w, n_samples), dtype=np.complex64)

    print(f"📊 FID dimensions: {fid_complex.shape}")
    print(f"⏱️ Acquisition duration: {time_points[-1]*1000:.1f} ms")
    print(f"🎵 Sampling frequency: {1/dwell_time/1000:.1f} kHz")

    # Simulate FID for each pixel
    for i in range(h):
        for j in range(w):
            if phantom[i, j] > 0.05:  # Only significant pixels
                # Initial amplitude (proton density)
                amplitude = phantom[i, j]

                # Frequency with spatial offset (gradient simulation)
                freq_offset = (i - h//2) * 10 + (j - w//2) * 8  # Hz
                larmor_freq = 1000 + freq_offset  # 64 MHz base (1.5T)

                # FID: exponential decay with precession
                decay = np.exp(-time_points / t2_star_map[i, j])
                oscillation = np.exp(1j * 2 * np.pi * larmor_freq * time_points)

                fid_complex[i, j, :] = amplitude * decay * oscillation

    # Add complex noise
    noise_real = np.random.normal(0, noise_level, fid_complex.shape)
    noise_imag = np.random.normal(0, noise_level, fid_complex.shape)
    noise_complex = noise_real + 1j * noise_imag

    fid_complex += noise_complex

    print(f"✅ FID simulated successfully")
    print(f"💾 Memory used: {fid_complex.nbytes / 1024**2:.1f} MB")

    return fid_complex, time_points

# Simulate FID
print("\n" + "="*60)
print("STEP 2: FID SIMULATION")
print("="*60)

fid_signals, time_array = simulate_fid_realistic(
    phantom,
    t2_star_map,
    te_time=80e-3,      # Typical TE for T2
    dwell_time=50e-6,   # 50 μs -> try increasing
    n_samples=1024,
    noise_level=0.005
)


fig, axes = plt.subplots(1, 2, figsize=(12, 5))

center_pixel_fid = fid_signals[128, 128, :]
axes[0].plot(time_array * 1000, np.abs(center_pixel_fid), 'b-', linewidth=2)
axes[0].set_title('3. FID (center pixel)')
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Magnitude')
axes[0].grid(True, alpha=0.3)

fid_cropped = center_pixel_fid[:550]  # First 550 samples -> better view of spiral
axes[1].plot(np.real(fid_cropped), np.imag(fid_cropped), 'r-', linewidth=2)
axes[1].set_title('4. FID Complex Plane')
axes[1].set_xlabel('Real Part')
axes[1].set_ylabel('Imaginary Part')
axes[1].grid(True, alpha=0.3)
axes[1].axis('equal')

plt.tight_layout()
plt.show()

print(f"📈 FID magnitude range: {np.abs(fid_signals).min():.4f} - {np.abs(fid_signals).max():.4f}")

## Module 4: From FID to K-SPACE (spatial encoding)

### Methods for FID → k-space conversion:
- direct method: Direct FFT (without going through FID)
- realistic method:
    1. Each TR repetition applies a different phase encoding gradient
    2. During readout, FID is acquired with frequency gradient
    3. FFT of FID → one k-space line
    4. Repeat for all phase encodings
- alternative method:
    1. At time t=0, FID represents non-decayed intensity -> this is equivalent to tissue intensity (signal before T2* decay)
    2. Apply 2D FFT to obtain k-space



In [ ]:
def phantom_to_kspace_direct(phantom):
    """
    Method 1: Direct phantom → k-space conversion

    This is the MOST CORRECT and simple way.
    In MRI, k-space is literally the 2D FFT of the image.
    """
    print("🎯 METHOD 1: DIRECT FFT")
    print("-" * 40)

    # 2D FFT with shift to center zero frequencies
    kspace_complex = fftshift(fft2(phantom))

    print(f"✅ K-space generated directly from phantom")
    print(f"📊 Shape: {kspace_complex.shape}")
    print(f"📊 Magnitude range: {np.abs(kspace_complex).min():.4f} - {np.abs(kspace_complex).max():.4f}")

    return kspace_complex

In [ ]:
def fid_to_kspace_realistic(fid_signals, phantom):
    """
    Method 2: Realistic physical simulation of MRI process

    This simulates how MRI acquisition REALLY works:
    1. Each TR repetition applies a different phase encoding gradient
    2. During readout, FID is acquired with frequency gradient
    3. FFT of FID → one k-space line
    4. Repeat for all phase encodings
    """
    print("🔬 METHOD 2: REALISTIC PHYSICAL SIMULATION")
    print("-" * 50)

    h, w, n_time = fid_signals.shape
    kspace_complex = np.zeros((h, w), dtype=np.complex128)

    print(f"📊 FID dimensions: {fid_signals.shape}")
    print(f"🎯 Target k-space: {kspace_complex.shape}")

    # Realistic simulation of MRI acquisition process
    for ky in range(h):  # For each phase encoding step
        print(f"  Processing k-space line {ky+1}/{h}", end='\r')

        # STEP 1: Simulate phase encoding gradient effect
        # In reality, this would modify the signal phase before acquisition
        phase_encoding_factor = np.exp(-1j * 2 * np.pi * ky * np.arange(w) / w)

        # STEP 2: Simulate FID acquisition during readout
        # Frequency gradient causes different x positions to precess at different frequencies
        readout_line = np.zeros(w, dtype=np.complex128)

        for kx in range(w):  # For each point in readout line
            if kx < n_time:  # Use available time samples
                # Sum contributions from all pixels along phase direction
                # weighted by their intensity and phase encoding
                pixel_contributions = 0
                for y_pos in range(min(h, n_time)):
                    if ky < h and kx < w:
                        # Contribution of pixel (ky, kx) to acquisition
                        # Includes: pixel intensity + phase encoding + temporal sampling
                        pixel_intensity = phantom[ky, kx] if kx < phantom.shape[1] else 0
                        phase_factor = phase_encoding_factor[kx]
                        time_sample = fid_signals[ky, kx, min(kx, n_time-1)]

                        pixel_contributions += pixel_intensity * phase_factor * time_sample

                readout_line[kx] = pixel_contributions
            else:
                readout_line[kx] = 0

        # STEP 3: FFT of readout line (simulates frequency encoding)
        kspace_line = fftshift(np.fft.fft(ifftshift(readout_line)))
        kspace_complex[ky, :] = kspace_line

    print(f"\n✅ K-space physically simulated from FID")
    print(f"📊 Magnitude range: {np.abs(kspace_complex).min():.4f} - {np.abs(kspace_complex).max():.4f}")

    return kspace_complex

In [ ]:
def fid_to_kspace_alternative(fid_signals):
    """
    Alternative FID → K-space conversion

    - Each FID contains spatially encoded information
    - K-space is the frequency representation of the spatial signal
    - We must extract spatial information from FID and then convert it

    """
    print("🔧 METHOD 3: ALTERNATIVE FID → K-SPACE CONVERSION")
    print("="*50)

    h, w, n_time = fid_signals.shape

    #print("📊 Method A: Initial FID amplitude")

    # Extract amplitude at time t=0 (first sample)
    spatial_image_from_fid = np.abs(fid_signals[:, :, 0])

    # Apply 2D FFT to obtain k-space
    kspace_method_a = fftshift(fft2(spatial_image_from_fid))

    print(f"   ✅ Image extracted: {spatial_image_from_fid.shape}")
    print(f"   📊 Range: {spatial_image_from_fid.min():.3f} - {spatial_image_from_fid.max():.3f}")

    return kspace_method_a, spatial_image_from_fid

In [ ]:
def fid_to_kspace_line_by_line(fid_signals, phantom, method='direct'):
    """
    Corrected version of original function

    Args:
        fid_signals: Simulated FID signals
        phantom: Original phantom
        method: 'direct', 'realistic', or 'alternative'
    """
    print(f"🔧 CORRECTION APPLIED - Method: {method}")
    print("=" * 50)

    if method == 'direct':
        # METHOD 1: Simplest and most correct way
        return phantom_to_kspace_direct(phantom)

    elif method == 'realistic':
        # METHOD 2: Complete physical simulation
        return fid_to_kspace_realistic(fid_signals, phantom)

    elif method == 'alternative':
        # METHOD 3: Simple alternative
        kspace_corrected, _ = fid_to_kspace_alternative(fid_signals)
        return kspace_corrected
        #return fid_to_kspace_alternative(fid_signals)

    else:
        raise ValueError("Method must be 'direct' or 'realistic'")

In [ ]:
# Convert FID to k-space
print("\n" + "="*60)
print("STEP 3: FID → K-SPACE")
print("="*60)

kspace_from_fid = fid_to_kspace_line_by_line(fid_signals, phantom, method='alternative')

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(np.log(np.abs(kspace_from_fid) + 1), cmap='hot')
axes[0].set_title('K-space from FID')
axes[0].axis('off')

center_line = np.abs(kspace_from_fid)[kspace_from_fid.shape[0]//2, :]
axes[1].plot(center_line, 'g-', linewidth=2)
axes[1].set_title('K-space Profile')
axes[1].set_xlabel('Spatial frequency')
axes[1].set_ylabel('Magnitude')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 📉 Exercise 2: Undersampling
#### Undersampling in MRI is a strategy to accelerate acquisition by reducing the amount of k-space data collected. Each k-space line requires a complete TR repetition and it must be considered that:
- Patients: Cannot remain still for too long
- Emergencies: Speed is needed for rapid diagnosis
- Comfort: Reduce claustrophobia and discomfort

Instead of acquiring all k-space lines, acquire only a percentage (70% for example).


In [ ]:
def apply_undersampling(kspace_complex, undersampling_factor=1.0):
    """Apply undersampling to k-space"""
    if undersampling_factor >= 1.0:
        return kspace_complex, np.ones(kspace_complex.shape, dtype=bool)

    print(f"🎯 APPLYING UNDERSAMPLING ({undersampling_factor:.1f}x)")

    h, w = kspace_complex.shape
    sampling_mask = np.zeros((h, w), dtype=bool)

    # Random sampling
    n_samples = int(h * w * undersampling_factor)
    flat_indices = np.random.choice(h * w, n_samples, replace=False)
    sampling_mask.flat[flat_indices] = True

    # Always keep the center (low frequencies)
    center_h, center_w = h // 2, w // 2
    center_region = 16
    for i in range(center_h - center_region, center_h + center_region):
        for j in range(center_w - center_region, center_w + center_region):
            if 0 <= i < h and 0 <= j < w:
                sampling_mask[i, j] = True

    # Apply mask
    kspace_sampled = kspace_complex * sampling_mask

    print(f"📊 Sampled points: {np.sum(sampling_mask)}/{h*w} ({100*np.sum(sampling_mask)/(h*w):.1f}%)")

    return kspace_sampled, sampling_mask

# Apply undersampling
kspace_undersampled, sampling_mask = apply_undersampling(kspace_from_fid, 0.7)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(sampling_mask, cmap='gray')
axes[0].set_title('Sampling Mask')
axes[0].axis('off')

axes[1].imshow(np.log(np.abs(kspace_undersampled) + 1), cmap='hot')
axes[1].set_title('Undersampled K-space')
axes[1].axis('off')

plt.tight_layout()
plt.show()

## Module 5: Image Reconstruction

In [ ]:
def reconstruct_from_kspace(kspace_complex):
    """Standard MRI reconstruction via inverse FFT"""
    print("🔄 IMAGE RECONSTRUCTION")
    print("-" * 40)

    # 2D inverse FFT
    reconstructed = ifft2(ifftshift(kspace_complex))
    reconstructed_magnitude = np.abs(reconstructed)

    print(f"✅ Image reconstructed: {reconstructed_magnitude.shape}")
    print(f"📊 Intensity range: {reconstructed_magnitude.min():.4f} - {reconstructed_magnitude.max():.4f}")

    return reconstructed_magnitude

# Reconstruction
print("\n" + "="*60)
print("STEP 4: RECONSTRUCTION")
print("="*60)

# Full reconstruction
reconstructed_full = reconstruct_from_kspace(kspace_from_fid)

# Reconstruction with zero-filling
kspace_zerofilled = kspace_undersampled.copy()
kspace_zerofilled[~sampling_mask] = 0
reconstructed_undersampled = reconstruct_from_kspace(kspace_zerofilled)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(reconstructed_full, cmap='gray')
axes[0].set_title('Complete Reconstruction')
axes[0].axis('off')

axes[1].imshow(reconstructed_undersampled, cmap='gray')
axes[1].set_title('Undersampled Reconstruction')
axes[1].axis('off')

plt.tight_layout()
plt.show()

## Module 6: Analysis and Visualization

In [ ]:
def visualize_complete_pipeline():
    """Visualize the complete MRI pipeline"""
    fig = plt.figure(figsize=(30, 16))

    # Layout: 3 rows x 4 columns
    gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)

    # Row 1: Phantom and maps
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.imshow(phantom, cmap='gray')
    ax1.set_title('1. T2-weighted Phantom')
    ax1.axis('off')

    ax2 = fig.add_subplot(gs[0, 1])
    im2 = ax2.imshow(t2_star_map * 1000, cmap='viridis')
    ax2.set_title('2. T2* Map (ms)')
    ax2.axis('off')
    plt.colorbar(im2, ax=ax2, label='T2* (ms)')

    # FID examples
    ax3 = fig.add_subplot(gs[0, 2])
    center_pixel_fid = fid_signals[128, 128, :]
    ax3.plot(time_array * 1000, np.abs(center_pixel_fid), 'b-', linewidth=2)
    ax3.set_title('3. FID (center pixel)')
    ax3.set_xlabel('Time (ms)')
    ax3.set_ylabel('Magnitude')
    ax3.grid(True, alpha=0.3)

    # FID in complex plane
    ax4 = fig.add_subplot(gs[0, 3])
    fid_cropped = center_pixel_fid[:150]  # First 150 samples -> better view of spiral
    ax4.plot(np.real(fid_cropped), np.imag(fid_cropped), 'r-', linewidth=2)
    ax4.set_title('4. FID Complex Plane')
    ax4.set_xlabel('Real Part')
    ax4.set_ylabel('Imaginary Part')
    ax4.grid(True, alpha=0.3)
    ax4.axis('equal')

    # Row 2: K-space
    ax5 = fig.add_subplot(gs[1, 0])
    ax5.imshow(np.log(np.abs(kspace_from_fid) + 1), cmap='hot')
    ax5.set_title('5. K-space from FID')
    ax5.axis('off')

    ax6 = fig.add_subplot(gs[1, 1])
    ax6.imshow(sampling_mask, cmap='gray')
    ax6.set_title('6. Sampling Mask')
    ax6.axis('off')

    ax7 = fig.add_subplot(gs[1, 2])
    ax7.imshow(np.log(np.abs(kspace_undersampled) + 1), cmap='hot')
    ax7.set_title('7. Undersampled K-space')
    ax7.axis('off')

    # K-space profile
    ax8 = fig.add_subplot(gs[1, 3])
    center_line = np.abs(kspace_from_fid)[kspace_from_fid.shape[0]//2, :]
    ax8.plot(center_line, 'g-', linewidth=2)
    ax8.set_title('8. K-space Profile')
    ax8.set_xlabel('Spatial frequency')
    ax8.set_ylabel('Magnitude')
    ax8.grid(True, alpha=0.3)

    # Row 3: Reconstructions
    ax9 = fig.add_subplot(gs[2, 0])
    ax9.imshow(reconstructed_full, cmap='gray')
    ax9.set_title('9. Complete Reconstruction')
    ax9.axis('off')

    ax10 = fig.add_subplot(gs[2, 1])
    ax10.imshow(reconstructed_undersampled, cmap='gray')
    ax10.set_title('10. Undersampled Reconstruction')
    ax10.axis('off')

    # Row 4: Quantitative analysis
    ax13 = fig.add_subplot(gs[2, 2])
    center_row = phantom.shape[0] // 2
    ax13.plot(phantom[center_row, :], 'b-', linewidth=3, label='Original Phantom')
    ax13.plot(reconstructed_full[center_row, :], 'g--', linewidth=2, label='Complete Reconstruction')
    ax13.plot(reconstructed_undersampled[center_row, :], 'r:', linewidth=2, label='Undersampled (70%)')
    ax13.set_title('11. Central Profiles Comparison')
    ax13.set_xlabel('Pixel')
    ax13.set_ylabel('Intensity')
    ax13.legend()
    ax13.grid(True, alpha=0.3)

    # Metrics
    ax14 = fig.add_subplot(gs[2, 3])
    metrics_full = calculate_mri_metrics(phantom, reconstructed_full)
    metrics_under = calculate_mri_metrics(phantom, reconstructed_undersampled)

    methods = ['Complete', 'Undersampled']
    mse_values = [metrics_full['MSE'], metrics_under['MSE']]
    psnr_values = [metrics_full['PSNR'], metrics_under['PSNR']]

    ax14_twin = ax14.twinx()
    bars1 = ax14.bar([x - 0.2 for x in range(len(methods))], mse_values, 0.4,
                     label='MSE', color='red', alpha=0.7)
    bars2 = ax14_twin.bar([x + 0.2 for x in range(len(methods))], psnr_values, 0.4,
                          label='PSNR (dB)', color='blue', alpha=0.7)

    ax14.set_xlabel('Reconstruction Method')
    ax14.set_ylabel('MSE', color='red')
    ax14_twin.set_ylabel('PSNR (dB)', color='blue')
    ax14.set_title('12. Quality Metrics')
    ax14.set_xticks(range(len(methods)))
    ax14.set_xticklabels(methods)

    plt.suptitle('Complete MRI Pipeline: Phantom → FID → K-space → Reconstruction',
                 fontsize=16, y=0.98)
    plt.show()

    # Print metrics
    print("\n📊 FINAL METRICS:")
    print("-" * 50)
    print(f"{'Method':<20} {'MSE':<12} {'PSNR (dB)':<12} {'SSIM':<10}")
    print("-" * 50)
    print(f"{'Complete':<20} {metrics_full['MSE']:<12.6f} {metrics_full['PSNR']:<12.2f} {metrics_full['SSIM']:<10.4f}")
    print(f"{'Undersampled':<20} {metrics_under['MSE']:<12.6f} {metrics_under['PSNR']:<12.2f} {metrics_under['SSIM']:<10.4f}")

# Final visualization
print("\n" + "="*60)
print("STEP 5: COMPLETE PIPELINE VISUALIZATION")
print("="*60)

visualize_complete_pipeline()

print("\n🎉 COMPLETE MRI PIPELINE EXECUTED!")
print("\n📚 Correct physical sequence implemented:")
print("   1️⃣ T2-weighted phantom with brain anatomy")
print("   2️⃣ Realistic FID simulation (decay + precession)")
print("   3️⃣ FID → K-space conversion (spatial encoding)")
print("   4️⃣ K-space undersampling")
print("   5️⃣ Reconstruction via inverse FFT")
print("   6️⃣ Quantitative analysis with metrics")

print("\n🔬 MRI concepts implemented:")
print("   • Tissue-specific T2* relaxation")
print("   • Complex Free Induction Decay")
print("   • Phase and frequency encoding")
print("   • Undersampling effects")
print("   • Zero-filling for incomplete data")
print("   • Image quality metrics")